# So you Want to Calculate an Impact Risk

Here we walk you through the steps necesssary to calculate the impact risk for a specified orbit, in this case 2024 YR4.
We're going to take a few steps:

1. Fetch the state vector for 2024 YR4 from JPL Horizons
2. Run calculate impacts, it will create and propagate samples through time and check for collisions.
3. Summarize the results into an impact risk.


Before we can start, you will need to install a couple packages.

```bash
pip install adam-core[plots] adam-assist
```


In [ ]:
# Fetch the state vector from Small Bodies Database (SBDB)
from adam_core.orbits.query import query_sbdb

orbit = query_sbdb(["2024 YR4"])

print(orbit.to_dataframe())

In [ ]:
# We need to know how many days out to run the propagation,
# relative to when the orbit state vector is defined.
# Let's go 30 days past the possible 2032 impact of 2024 YR4
from adam_core.time import Timestamp
approx_impact_date = Timestamp.from_iso8601(["2032-12-22"], scale="tdb")
thirty_days_after_impact = approx_impact_date.add_days(30)
days_until_thirty_days_after_impact, _ = thirty_days_after_impact.difference(orbit.coordinates.time)
print(days_until_thirty_days_after_impact)

In [ ]:
# Now we initialize our propagator, and pass it to our calculate_impacts function
from adam_core.dynamics.impacts import calculate_impacts
from adam_assist import ASSISTPropagator

propagator = ASSISTPropagator()

variants, impacts = calculate_impacts(
    orbit,
    days_until_thirty_days_after_impact[0].as_py(),
    propagator,
    num_samples=10000,
    processes=10, # Multiprocessing speeds things up if you have the CPUs
)

print(variants, impacts)

In [ ]:
# Now we can summarize the returns, a simple ratio in our case
# with only 1 orbit considered
from adam_core.dynamics.impacts import calculate_impact_probabilities
ip = calculate_impact_probabilities(variants, impacts)
print(ip.to_dataframe())

We've recently added some visualization functionality to visualize both the risk corridor and the impact events. Let's first look at the impact corridor.

In [ ]:
from adam_core.dynamics.plots import plot_risk_corridor

fig = plot_risk_corridor(impacts, title="Risk Corridor for 2024 YR4")
fig.show()

In [6]:
from adam_core.dynamics.plots import generate_impact_visualization_data

propagated_best_fit_orbit, propagated_variants = generate_impact_visualization_data(
    orbit,
    variants,
    impacts,
    propagator,
    time_step=5,
    time_range=120,
    max_processes=None
)

In [ ]:
from adam_core.dynamics.plots import plot_impact_simulation

fig = plot_impact_simulation(
    propagated_best_fit_orbit, 
    propagated_variants, 
    impacts, 
    grid=True, 
    title="2024 YR4 Impact Simulation",
)
fig.show()

And there you have it. 